In [30]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kyeul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes')) # random_state로 매번 동일하게 shuffle한다.
documents = dataset.data
print('샘플 수: ', len(documents))

샘플 수:  11314


In [37]:
news_df

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased whati disagr...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hardat...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...",danny rubenstein israeli journalist will speak...
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree home runs clemens always memorable kinda...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...


In [36]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거 # clean_doc이라는 새로운 column에 저장
# news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") #오.. DataFrame에 replace로 치환할 수 있구나. 늘 re썼는데 몰랐네. #알파벳을 제외하고 모두 제거
news_df['clean_doc'] = news_df['document'].apply(lambda x : re.sub('[^a-zA-Z ]', '', x))
# news_df['clean_doc'] = news_df['document'].str.replace("^", "")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거 ) # 이걸 왜 제거하는 거지?
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))  # 낱말이 3개 미만인 것을 제거한다. 
# 소문자로 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
# nltk로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 단어로 쪼개서 토큰화함
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if len(item)!= 0])


In [38]:
len(tokenized_doc)

11314

# 본격적인 LDA 예제 시작

In [39]:
type(tokenized_doc[0]) # 데이터를 한번 살펴보자

list

In [40]:
# gensim의 corpora를 사용해서 단어 빈도를 구해보자
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]


In [41]:
print(tokenized_doc[0])
print(corpus[0])

['well', 'sure', 'story', 'seem', 'biased', 'whati', 'disagree', 'statement', 'media', 'toruin', 'israels', 'reputation', 'rediculous', 'media', 'isthe', 'proisraeli', 'media', 'world', 'lived', 'europei', 'realize', 'incidences', 'described', 'theletter', 'occured', 'media', 'whole', 'seem', 'toignore', 'subsidizing', 'israels', 'existance', 'theeuropeans', 'least', 'degree', 'thinkthat', 'might', 'reason', 'report', 'clearly', 'theatrocitieswhat', 'shame', 'austria', 'daily', 'reports', 'ofthe', 'inhuman', 'acts', 'commited', 'israeli', 'soldiers', 'blessingreceived', 'government', 'makes', 'holocaust', 'guiltgo', 'away', 'look', 'jews', 'treating', 'raceswhen', 'power', 'unfortunate']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 4), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1

In [42]:
for c, t in  zip(corpus[0], tokenized_doc[0]):
    print(c, t)

(0, 1) well
(1, 1) sure
(2, 1) story
(3, 1) seem
(4, 1) biased
(5, 1) whati
(6, 1) disagree
(7, 1) statement
(8, 1) media
(9, 1) toruin
(10, 1) israels
(11, 1) reputation
(12, 1) rediculous
(13, 1) media
(14, 1) isthe
(15, 1) proisraeli
(16, 1) media
(17, 1) world
(18, 1) lived
(19, 2) europei
(20, 1) realize
(21, 1) incidences
(22, 1) described
(23, 1) theletter
(24, 1) occured
(25, 1) media
(26, 4) whole
(27, 1) seem
(28, 1) toignore
(29, 1) subsidizing
(30, 1) israels
(31, 1) existance
(32, 1) theeuropeans
(33, 1) least
(34, 1) degree
(35, 1) thinkthat
(36, 1) might
(37, 1) reason
(38, 1) report
(39, 2) clearly
(40, 1) theatrocitieswhat
(41, 1) shame
(42, 1) austria
(43, 1) daily
(44, 1) reports
(45, 1) ofthe
(46, 1) inhuman
(47, 1) acts
(48, 1) commited
(49, 1) israeli
(50, 1) soldiers
(51, 1) blessingreceived
(52, 1) government
(53, 1) makes
(54, 1) holocaust
(55, 1) guiltgo
(56, 1) away
(57, 1) look


In [43]:
for c, t in  zip(corpus[1], tokenized_doc[1]):
    print(c, t)

(54, 1) yeah
(58, 1) expect
(59, 1) people
(60, 1) read
(61, 1) actually
(62, 1) accept
(63, 1) hardatheism
(64, 1) need
(65, 1) little
(66, 1) leap
(67, 1) faith
(68, 2) jimmy
(69, 1) logic
(70, 1) runs
(71, 1) outof
(72, 1) steamjimsorry
(73, 1) cant
(74, 1) pity
(75, 1) sorry
(76, 1) feelings
(77, 1) ofdenial
(78, 1) faith
(79, 1) need
(80, 2) well
(81, 1) pretend
(82, 1) willall
(83, 1) happily
(84, 1) ever
(85, 1) anyway
(86, 1) maybe
(87, 1) start
(88, 1) newsgroupaltatheisthard
(89, 1) wont
(90, 1) bummin
(91, 1) muchbyebye
(92, 1) dont
(93, 1) forget
(94, 1) flintstones
(95, 1) chewables


In [44]:
dictionary[66]

'ever'

In [45]:
len(dictionary)

152981

# LDA 모델 훈련시키기

In [46]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

(0, '0.012*"bike" + 0.010*"engine" + 0.006*"miles" + 0.006*"cars"')
(1, '0.042*"space" + 0.012*"nasa" + 0.012*"launch" + 0.009*"satellite"')
(2, '0.010*"would" + 0.009*"people" + 0.005*"many" + 0.005*"believe"')
(3, '0.006*"ships" + 0.006*"lemieux" + 0.004*"naval" + 0.004*"navy"')
(4, '0.009*"available" + 0.009*"information" + 0.008*"file" + 0.008*"program"')
(5, '0.004*"bone" + 0.003*"amps" + 0.003*"nettersi" + 0.003*"smokers"')
(6, '0.011*"gordon" + 0.010*"banks" + 0.009*"surrender" + 0.009*"skepticism"')
(7, '0.016*"said" + 0.010*"went" + 0.010*"armenians" + 0.008*"armenian"')
(8, '0.009*"radar" + 0.008*"filename" + 0.006*"detector" + 0.006*"slave"')
(9, '0.015*"entries" + 0.009*"wire" + 0.006*"cover" + 0.006*"wiring"')
(10, '0.014*"widget" + 0.007*"visual" + 0.006*"xlib" + 0.006*"smokeless"')
(11, '0.013*"power" + 0.010*"cable" + 0.008*"ground" + 0.007*"water"')
(12, '0.004*"capitol" + 0.003*"tvtwm" + 0.002*"cassette" + 0.002*"washer"')
(13, '0.010*"government" + 0.008*"people" + 0

In [47]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.012*"bike" + 0.010*"engine" + 0.006*"miles" + 0.006*"cars" + 0.005*"henrik" + 0.005*"bikes" + 0.005*"road" + 0.004*"honda" + 0.004*"azeri" + 0.004*"azeris"')
(1, '0.042*"space" + 0.012*"nasa" + 0.012*"launch" + 0.009*"satellite" + 0.008*"shuttle" + 0.007*"lunar" + 0.007*"orbit" + 0.007*"earth" + 0.007*"flight" + 0.006*"mission"')
(2, '0.010*"would" + 0.009*"people" + 0.005*"many" + 0.005*"believe" + 0.005*"think" + 0.004*"even" + 0.004*"must" + 0.004*"dont" + 0.004*"also" + 0.004*"jesus"')
(3, '0.006*"ships" + 0.006*"lemieux" + 0.004*"naval" + 0.004*"navy" + 0.003*"toshiba" + 0.003*"juneau" + 0.003*"disc" + 0.002*"inkjet" + 0.002*"winmarks" + 0.002*"seminar"')
(4, '0.009*"available" + 0.009*"information" + 0.008*"file" + 0.008*"program" + 0.007*"data" + 0.006*"number" + 0.006*"also" + 0.005*"system" + 0.004*"list" + 0.004*"send"')
(5, '0.004*"bone" + 0.003*"amps" + 0.003*"nettersi" + 0.003*"smokers" + 0.002*"methanks" + 0.002*"floptical" + 0.002*"cigarettes" + 0.001*"epson" + 0.

In [48]:
import pyLDAvis.gensim_models

In [49]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [50]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(2, 0.16877286), (7, 0.027221393), (13, 0.2569569), (14, 0.5327135)]
1 번째 문서의 topic 비율은 [(2, 0.28377843), (6, 0.03161275), (10, 0.042965397), (13, 0.09242617), (14, 0.49633712), (18, 0.031635765)]
2 번째 문서의 topic 비율은 [(7, 0.063389905), (13, 0.26672918), (14, 0.6546383)]
3 번째 문서의 topic 비율은 [(2, 0.1378609), (4, 0.28919023), (14, 0.36738828), (16, 0.11672664), (17, 0.0153438775), (19, 0.06196217)]
4 번째 문서의 topic 비율은 [(14, 0.5600072), (15, 0.4008318)]


In [54]:
ldamodel.per_word_topics

False

In [51]:
# 토픽 번호와 해당 문서에서 차지하는 분포도를 잘 볼 수 있도록 출력

In [92]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list # ldamodel.per_word_topics 이게 뭐할 때 쓰는거지? 
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해 비중이 높은 토픽순으로 정렬함

        for j, (topic_num, prop_topic) in enumerate(doc): # 몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:
                df = pd.DataFrame([[int(topic_num), round(prop_topic,4), topic_list]])
                topic_table = pd.concat([topic_table, df], ignore_index=True)
            else:
                break
    return(topic_table)


In [97]:
topictable = make_topictable_per_doc(ldamodel, corpus)


In [100]:
topictable = topictable.reset_index(drop=True)
topictable.columns = ['가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,14,0.5323,"[(2, 0.16927293), (7, 0.026868861), (13, 0.257..."
1,14,0.4963,"[(2, 0.28377613), (6, 0.03161275), (10, 0.0429..."
2,14,0.6546,"[(7, 0.063413024), (13, 0.26672024), (14, 0.65..."
3,14,0.3673,"[(2, 0.13788955), (4, 0.2891885), (14, 0.36733..."
4,14,0.5600,"[(14, 0.5600082), (15, 0.40083072)]"
5,2,0.3964,"[(2, 0.39637366), (11, 0.23741533), (14, 0.325..."
6,19,0.3910,"[(2, 0.2147679), (10, 0.10566796), (14, 0.2219..."
7,14,0.5102,"[(0, 0.07899012), (2, 0.20126143), (7, 0.04174..."
8,14,0.6623,"[(4, 0.08552919), (10, 0.10407432), (14, 0.662..."
9,14,0.6483,"[(0, 0.086973585), (2, 0.1515842), (9, 0.10064..."
